# Ta sẽ sử dụng kĩ thuật Collaboration filtering để xây dựng hệ thống đề xuất sách dựa trên sự đánh giá của người dùng với các cuốn sách đã đọc.

## 1.Import các thư viện cần thiết

In [2]:
import numpy as np
import pandas as pd

Ta sẽ sử dụng bộ dữ liệu về sách, người đọc và điểm đánh giá về cuốn sách ở trên Kaggle.

Link data: https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset

In [3]:
book_df = pd.read_csv("Books.csv")
rate_df = pd.read_csv("Ratings.csv")

C:\Users\HP\AppData\Local\Temp\ipykernel_8244\2642730539.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  book_df = pd.read_csv("Books.csv")


## 2 Khám phá dữ liệu

### 2.1 Đầu tiên, ta coi bảng dữ liệu về sách

#### 2.1.1 Các dòng dữ liệu

In [4]:
book_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


#### 2.1.2 Số dòng số cột

In [5]:
book_df.shape

(271360, 8)

Dữ liệu có 271360 dòng và 8 cột

#### 2.1.3 Các cột dữ liệu và ý nghĩa

In [6]:
book_df.dtypes

ISBN                   object
Book-Title             object
Book-Author            object
Year-Of-Publication    object
Publisher              object
Image-URL-S            object
Image-URL-M            object
Image-URL-L            object
dtype: object

##### Ý nghĩa các cột:

- **ISBN**: ID của sách
- **Book-Title**: Tên sách
- **Book-Author**: Tác giả
- **Year-Of-Publication**: Năm xuất bản
- **Publisher**: Nhà xuất bản
- **Image**: Ảnh sách

#### 2.1.4 Dữ liệu thiếu và trùng

In [7]:
book_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


Dữ liệu có giá trị nan ở cột Book-Author và Publisher nhưng vì ít nên ta bỏ qua

In [8]:
book_df.duplicated().sum()

0

Không có dòng trùng

### 2.2 Tiếp theo, ta coi bảng dữ liệu về đánh giá sách

#### 2.2.1 Các dòng dữ liệu

In [9]:
rate_df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


#### 2.2.2 Số dòng số cột

In [10]:
rate_df.shape

(1149780, 3)

Dữ liệu có 1149780 dòng và 3 cột

#### 2.1.3 Các cột dữ liệu và ý nghĩa

In [11]:
rate_df.dtypes

User-ID         int64
ISBN           object
Book-Rating     int64
dtype: object

##### Ý nghĩa các cột:

- **User-ID**: ID của người dùng
- **ISBN**: ID sách
- **Book-Rating**: Điểm đánh giá của người dùng về sách

#### 2.1.4 Dữ liệu thiếu và trùng

In [12]:
rate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


Dữ liệu không có giá trị nan

In [13]:
rate_df.duplicated().sum()

0

Dữ liệu không có dòng trùng

### 3 Tạo hệ thống đề xuất sách

Để tạo nên một hệ thống đề xuất, chúng cần ta một bảng data khác có dòng i là cuốn sách thứ i, cột j là người dùng thư và các ô (i,j) có giá trị là điểm đánh giá của người j về sách. Sở dĩ ta cần làm điều này vì để xây dựng hệ thống đề xuất, ta cần thêm một số thuộc tính cho các cuốn sách, vậy nên bên cạnh điểm đánh giá của độc giả, sách sẽ được thêm vài feature để có thể đề xuất được

### 3.1 Xây dựng bảng này từ rate_df

Vì thiết bị không cho phép nên ta chỉ lấy 10000 dòng dữ liệu ở rate_df để chuyển đổi

In [14]:
df_rate = rate_df[:10000].pivot(index ='ISBN',columns='User-ID',values='Book-Rating')

In [15]:
df_rate.shape

(9340, 941)

Ta đã có bảng dữ liệu mới gồm 9340 cuốn sách và 941 người đánh giá

In [16]:
df_rate.head()

User-ID,2,7,8,9,10,12,14,16,17,19,...,278832,278836,278838,278843,278844,278846,278849,278851,278852,278854
ISBN,,,,,,,,,,,,,,,,,,,,,
0002005018,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0002231115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0002232766,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0002240114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000225669X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 3.2 Tạo bảng cho biết người dùng j có đánh giá phim i không

Bảng này sẽ có ô (i,j) có giá trị 1 nếu người j có đánh giá sách i, có giá trị 0 nếu người j không đánh giá

In [17]:
R_df = 1 - df_rate.isnull()
R_df.head()


User-ID,2,7,8,9,10,12,14,16,17,19,...,278832,278836,278838,278843,278844,278846,278849,278851,278852,278854
ISBN,,,,,,,,,,,,,,,,,,,,,
0002005018,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0002231115,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0002232766,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0002240114,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
000225669X,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Ta sẽ điền điểm của những cuốn sách chưa được người đọc đánh giá là -1

In [18]:
df_rate.fillna(-1,inplace=True)
df_rate.head()

User-ID,2,7,8,9,10,12,14,16,17,19,...,278832,278836,278838,278843,278844,278846,278849,278851,278852,278854
ISBN,,,,,,,,,,,,,,,,,,,,,
0002005018,-1.0,-1.0,5.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
0002231115,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
0002232766,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
0002240114,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
000225669X,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


 Ta sẽ thêm 10 thuộc tính của sách. Lưu các giá trị của thuộc tính này vào bảng X. Mỗi dòng của X là các giá trị này

In [19]:
X = np.ones((df_rate.shape[0],10))


X.shape

(9340, 10)

Tạo W là ma trận tham số của từng người dùng khi đánh giá phim

In [20]:
W = np.ones((df_rate.shape[1],10))


W.shape

(941, 10)

In [21]:
b = np.ones((1,df_rate.shape[1]))

Tính hàm cost function

$$ J = \frac{1}{2} \sum\limits_{j=0}^{n_u-1} \sum\limits_{i=0}^{n_m-1} r(i,j) (\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)} - y^{(i,j)})^2 + \text{regularization} $$

Hàm cost function tự thiết lập

In [22]:
def cos_func(X,W,b,R_df,df_rate,lambda_reg):
    # Số người đánh giá
    nu = df_rate.shape[1]
    # Số bộ phim
    nm = df_rate.shape[0]
    #Khởi tạo giá trị J
    J = 0
    for j in range(nu):
        for i in range(nm):
            #r(i,j)
            r = 1-R_df[i,j]
            if r == 0:
                continue 
            # wj
            w_j = W[j]
            
            # xi
            x_i = X[i]
            
            # bj
            b_j = b[j]
            J += r*(np.dot(w_j,x_i)+b_j - df_rate[i,j])**2
           
           
    J /= 2
    #Cong them hang so
    W_square = np.square(W)
    X_square = np.square(X)

    J += (lambda_reg/2)*(np.sum(W_square)+np.sum(X_square))

    return J

In [23]:
import tensorflow as tf
from tensorflow import keras

Thực hiện hàm cost function bằng vectorization để tối ưu tốc độ

In [24]:
def vec_cos_func(X,W,b,R_df,df_rate,lambda_reg):
    j = R_df*(tf.linalg.matmul(X,tf.transpose(W)) + b - df_rate)
    J = 0.5*tf.reduce_sum(j**2) + (lambda_reg/2)*(tf.reduce_sum(W**2)+tf.reduce_sum(X**2))
    return J

Khởi tạo các trọng số W,X,b:

- W là trọng số của người dùng tương ứng với các feature của bộ phim. Wj sẽ là bộ trọng số của người dùng j.
- X là bộ giá trị feature của các cuốn sách. Xi sẽ là bộ giá trị của các feature của cuốn sách i.

In [25]:
num_movies, num_users = df_rate.shape
num_features = 10

# Set Initial Parameters (W, X), use tf.Variable to track these variables
tf.random.set_seed(1234) # for consistent results
W = tf.Variable(tf.random.normal((num_users,  num_features),dtype=tf.float64),  name='W')
X = tf.Variable(tf.random.normal((num_movies, num_features),dtype=tf.float64),  name='X')
b = tf.Variable(tf.random.normal((1,          num_users),   dtype=tf.float64),  name='b')

# Instantiate an optimizer.
optimizer = keras.optimizers.Adam(learning_rate=1e-1)

Ta sẽ train hệ thống này 

In [26]:
iterations = 50
lambda_ = 1
for iter in range(iterations):
    # Use TensorFlow’s GradientTape
    # to record the operations used to compute the cost 
    with tf.GradientTape() as tape:

        # Compute the cost (forward pass included in cost)
        cost_value = vec_cos_func(X, W, b,  R_df.values,df_rate.values, lambda_)

    # Use the gradient tape to automatically retrieve
    # the gradients of the trainable variables with respect to the loss
    grads = tape.gradient( cost_value, [X,W,b] )

    # Run one step of gradient descent by updating
    # the value of the variables to minimize the loss.
    optimizer.apply_gradients( zip(grads, [X,W,b]) )

    # Log periodically.
    if iter % 10 == 0:
        print(f"Training loss at iteration {iter}: {cost_value:0.1f}")

Training loss at iteration 0: 177373.2
Training loss at iteration 10: 33646.9
Training loss at iteration 20: 18540.0
Training loss at iteration 30: 12369.9
Training loss at iteration 40: 9077.7


Ta chọn ngẫu nhiên một người dùng để xem mô hình có chạy chính xác với thực tế không. Ví dụ người dùng ở cột thứ 600 trong df_rate

In [27]:
user = df_rate.iloc[:,600]

user[user!=-1]

ISBN
0060928336    0.0
0316601950    6.0
034543479X    8.0
0374480095    8.0
0446364495    0.0
0446670251    6.0
0804115761    7.0
0821766570    5.0
1551669110    6.0
Name: 278007, dtype: float64

Lấy ra danh sách các cuốn sách đã đọc

In [28]:
had_rated = R_df.loc[R_df.iloc[:,600] == 1].index
had_rated = R_df.index.get_indexer(had_rated)
had_rated

array([ 180, 1246, 1478, 2422, 3803, 3929, 6829, 7130, 7747], dtype=int64)

Tính điểm dự đoán dự kiến của người dùng này với các cuốn sách và lấy ra những cuốn đã đọc để so sánh với mô hình

In [29]:
user_pred = np.matmul(X.numpy(),np.transpose(W[600].numpy()))+b[0,600].numpy()

for i in had_rated:
    print('Thuc te {}'.format(df_rate.iloc[i,600]))
    print('Du doan {}'.format(user_pred[i]))
    print('-----')

Thuc te 0.0
Du doan 0.1880906447053179
-----
Thuc te 6.0
Du doan 6.279184545192408
-----
Thuc te 8.0
Du doan 7.582568102564035
-----
Thuc te 8.0
Du doan 7.655339187533726
-----
Thuc te 0.0
Du doan 0.11031467687829011
-----
Thuc te 6.0
Du doan 5.5961770537205435
-----
Thuc te 7.0
Du doan 6.684877672218728
-----
Thuc te 5.0
Du doan 4.72491700040362
-----
Thuc te 6.0
Du doan 5.450627724306647
-----


Ta thấy điểm không lệch quá nhiều

Từ đây ta sẽ xây dựng một list sách gồm 20 cuốn mới cho người dùng thứ 600 này

Ta sẽ thêm cột IBSN để lấy index của sách bên cạnh cột dự đoán

In [30]:
recommend_book_df = pd.DataFrame(index=df_rate.index,columns = ['Rating_pre'],data=user_pred)
recommend_book_df['Has_rated'] = R_df.iloc[:,600]
recommend_book_df.head()

,Rating_pre,Has_rated
ISBN,,
0002005018,1.377852,0
0002231115,1.608569,0
0002232766,1.801957,0
0002240114,1.945322,0
000225669X,4.218050,0


Đầu tiên, ta bỏ những cuốn sách đã được đánh giá

In [31]:
recommend_book_df.drop(recommend_book_df[recommend_book_df['Has_rated']==1].index,inplace =True)

Sau đó, sắp xếp điểm theo thứ tự giảm dần

In [32]:
recommend_book_df.sort_values(by = 'Rating_pre',ascending = False,inplace = True)

Sau đó, để lấy tên sách, ta sẽ gộp bảng này và book_df thông qua thuộc tính ISBN

In [33]:
merge_book = pd.merge(recommend_book_df,book_df,on='ISBN',how='inner')

Ta sẽ đề xuất 20 cuốn sách được dự đoán là được đánh giá cao nhất bởi người dùng này 

In [34]:
print("List sách đề xuất")
merge_book[:20]['Book-Title']

List sách đề xuất


0                                          Color Purple
1          I Know This Much Is True (Oprah's Book Club)
2     The Art of Possibility: Transforming Professio...
3                                       Into the Forest
4                 She's Come Undone (Oprah's Book Club)
5                                 Bridget Jones's Diary
6                                     The Street Lawyer
7                   Wild Mind: Living the Writer's Life
8                        How Stella Got Her Groove Back
9                                     Step Off the Edge
10                                     Die zweite Haut.
11    Cadillac Jukebox (Dave Robicheaux Mysteries (P...
12                          EVEN COWGIRLS GET THE BLUES
13                         What You Say Is What You Get
14                                Piercing the Darkness
15                                     Golf Instruction
16                                  The yawning heights
17                                              